In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, SimpleRNN, LSTM, GRU, Dense

In [2]:
num_words= 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

In [6]:
maxlen = 100
x_train = pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=maxlen, padding='post', truncating='post')

In [4]:
version_fn = getattr(tf.keras, "version", None)
if version_fn and version_fn().startswith("3."):
    import tensorflow.keras as keras
else:
    keras = tf.keras

In [6]:
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.models import Model
embedding =100
#encoder_input = Input(shape=(maxlen,))
#model = Sequential([
 #   Embedding(input_dim=num_words, output_dim=embedding, input_length=maxlen),
 #   Bidirectional(LSTM(5)),
 #   Dense(1, activation='sigmoid')
#])
input_layer = Input(shape=(maxlen,))
x = Embedding(input_dim=num_words, output_dim=embedding)(input_layer)
x = Bidirectional(LSTM(5, return_sequences=False))(x)
x = Dropout(0.2)(x)
output_layer = Dense(1, activation='sigmoid')(x)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train,batch_size=num_words, epochs=1)
model.build(input_shape=(1,maxlen))


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 677ms/step - accuracy: 0.5274 - loss: 0.6922


In [7]:
dummy_input = tf.zeros((1, maxlen))

# Call the model with dummy input to initialize it
output = model(dummy_input)

# Print the model summary to verify
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 100, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 10)             │         4,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,012,755 (11.49 MB)

 Trainable params: 1,004,251 (3.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008,504 (7.66 MB)

In [8]:
train_loss, train_acc = model.evaluate(x_test, y_test, verbose=2)

782/782 - 3s - 4ms/step - accuracy: 0.5768 - loss: 0.6899


In [9]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [10]:
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [11]:
log_dir

'logs/fit20240726-013333'

In [12]:
model_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [13]:
import mlflow
earLy_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [14]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, callbacks=[model_callbacks, earLy_stop]
)

Epoch 1/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.7095 - loss: 0.5687 - val_accuracy: 0.8068 - val_loss: 0.4246
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.8692 - loss: 0.3233 - val_accuracy: 0.8020 - val_loss: 0.4313
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.9171 - loss: 0.2320 - val_accuracy: 0.8039 - val_loss: 0.4575
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.9414 - loss: 0.1674 - val_accuracy: 0.7886 - val_loss: 0.5441
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.9567 - loss: 0.1260 - val_accuracy: 0.7905 - val_loss: 0.6027
Epoch 6/100
613/782 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.9692 - loss: 0.0899

KeyboardInterrupt: 

In [15]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 100, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 10)             │         4,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,012,755 (11.49 MB)

 Trainable params: 1,004,251 (3.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008,504 (7.66 MB)

In [16]:
from mlflow.models.signature import infer_signature
model_sign = infer_signature(x_test, model.predict(x_test))

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


In [17]:
exp_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
exp_name= "IMDB_bidirection_lstm_time:" + exp_timestamp
#exp_name = "IMDB_bidirection_lstm"
run_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_name = "IMDB_bidirection_lstm_run:" + run_timestamp

In [18]:
def tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign):
    mlflow.set_experiment(exp_name)
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("batch_size", num_words )
        mlflow.log_param("learning_rate", 0.001)
        mlflow.log_param("epochs", 100)
        mlflow.log_metric("train_loss", train_loss)
        mlflow.log_metric("train_accuracy", train_acc)
        mlflow.tensorflow.log_model(model, "IMDB", signature=model_sign)
    mlflow.end_run()

In [19]:
tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign)

2024/07/26 01:36:19 INFO mlflow.tracking.fluent: Experiment with name 'IMDB_bidirection_lstm_time:20240726-013618' does not exist. Creating a new experiment.


In [ ]:
model.export(filepath='saved_model', format='tf')
model.save('models/BI_LSTM_model.h5')
model.save_weights('models/LSTM.weights.h5')

INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


Saved artifact at 'saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100), dtype=tf.float32, name='keras_tensor_50')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1899094009296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1899094003344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1899094014480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1899094015632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1899094011408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1899094015248: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
import bentoml

bento_model = bentoml.tensorflow.save_model("bento_model", model)

INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmponxewvsebentoml_model_bento_model\assets


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmponxewvsebentoml_model_bento_model\assets
